In [1]:
import asyncio
import httpx
import pandas as pd
import secret
import refresh
import requests
from time import perf_counter

In [2]:
id_token = refresh.refreshed()
headers = {"content-type":"application/json", "x-api-key": secret.api_key, "id-token": id_token}

In [3]:
df = pd.read_csv('us_chinese_contact.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2731 entries, 0 to 2730
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   vinny_id  2731 non-null   int64
dtypes: int64(1)
memory usage: 21.5 KB


In [ ]:
async def get_result(vinny_id:int):
    async with httpx.AsyncClient() as client:
        r = await client.get(f'https://bps-world.vincere.io/api/v2/candidate/{vinny_id}', headers=headers)
        results.append(r.json())
        return
    
async def main():
    tasks = []
    gen_id = df['vinny_id']
    for vinny_id in gen_id:
        tasks.append(get_result(vinny_id))
    await asyncio.gather(*tasks)

results = []

In [ ]:
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')

In [ ]:
results

In [ ]:
def get_data(id:int):
    r = requests.get(f'https://bps-world.vincere.io/api/v2/candidate/{id}', headers=headers)
    data_container.append(r.json())
    return

def main():
    for i in df['vinny_id']:
        get_data(i)
    return

data_container=[]

In [ ]:
start_slow = perf_counter()
main()
stop_slow = perf_counter()
print('time taken:', stop_slow - start_slow, ' seconds')

In [ ]:
data_container

## Updating contacts faster way (custom fields)

### Requirements for update
{
"data":[{'field_key':'', 'field_value':'',
}] 
}

In [4]:
# async def get_data(vinny_id:int) -> dict:
#     row = df.loc[df['vinny_id'] == vinny_id]
#     data = await {'data':[{
#         'field_value_ids':[11513],
#         'field_values':[2]
#     }]}
#     async for col, val in row.items():
#         async for value in val:
#             await data['data'].append({'field_key':col,'field_value':value})
#     return data

data = {'data':[{
    'field_key':'00e954f748e5e5fde7e341cb2697953d',
    'field_value_ids': [11513],
    'field_values':[2]
}]}

sem = asyncio.Semaphore(100)

async def update_candidate(vinny_id:int):
    async with sem:
        async with httpx.AsyncClient() as client:
            r = await client.put(f'https://bps-world.vincere.io/api/v2/candidate/{vinny_id}/customfields', headers=headers, json=data)
            results.append(r.json())
            return

async def main():
    tasks = []
    gen_id = df['vinny_id']
    for vinny_id in gen_id:
        tasks.append(update_candidate(vinny_id))
    await asyncio.gather(*tasks, return_exceptions=True)

results = []

In [ ]:
start = perf_counter()
await main()
stop = perf_counter()
print('time taken:', stop - start, ' seconds')